# Using Pandas (basic introduction)
Pandas is a library to work with data using relational tables

To prepare for this lesson execute the following cell

In [1]:
!git clone https://github.com/datasciencedojo/datasets.git

Cloning into 'datasets'...


import the pandas library and assign it a shorter alias

In [2]:
import pandas as pd

## Loading data
Pandas includes a rich set of input functions that allow you to get data from various file types

| function | format | notes |
|----------|--------|-------|
| `pd.read_csv` | textual csv | |
| `pd.read_excel` | binary excel format | requires external library |
| `pd.read_parquet` | fast binary columnar format| requires pyarrow |

A data frame contains many functions to explore it e.g. the `.head()` method shows the first lines of a data frame

In [43]:
titanic = pd.read_csv("datasets/titanic.csv")
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Projection, Selection and Extension
a data frame is a table; you can get its column names using the `.columns` attributes

In [5]:
titanic.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

columns can be accessed individually or in groups; this operation is called **projection**

Single columns can be accessed either 
1. using a square bracket operator `df.["age"]`
2. using the dot operator if the column name is a good **identifier** `df.age`

Each column is called a **Series** in pandas jargon

Groups of columns can be accessed by passing a list of strings to the bracket operator 

In [6]:
titanic[["Survived","Pclass","Sex","Age"]].head()

,Survived,Pclass,Sex,Age
0,0,3,male,22.0
1,1,1,female,38.0
2,1,3,female,26.0
3,1,1,female,35.0
4,0,3,male,35.0
...,...,...,...,...
886,0,2,male,27.0
887,1,1,female,19.0
888,0,3,female,NaN
889,1,1,male,26.0


operations on series are vectorized i.e. each individual element is used to get a new vector

Operations within a series and a scalar value are repeated for all values of a series

```python
titanic.Pclass == 1
```

returns a series of booleans

By passing a list of booleans to the square bracket operators this filters all of the lines which are satisfying the logic statement expressed; this operation is called **selection** which is a synonim for filter

In [8]:
titanic[titanic.Pclass == 1].head()

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.000000,1,0,PC 17599,71.283300,C85,C
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.100000,C123,S
7,0,1,"McCarthy, Mr. Timothy J",male,54.000000,0,0,17463,51.862500,E46,S
12,1,1,"Bonnell, Miss. Elizabeth",female,58.000000,0,0,113783,26.550000,C103,S
24,1,1,"Sloper, Mr. William Thompson",male,28.000000,0,0,113788,35.500000,A6,S


Usually selection and projection are used at the same time; the `.loc[,]` operator can be conveniently used for this purpose; its arguments are:
1. a boolean list for rows or the splice operator `:` for no filter
2. a string list of column names or the splice operator `:` for all columns

In [10]:
titanic.loc[titanic.Pclass==1,["Survived","Sex","Age"]].head()

Survived,Sex,Age
1,female,38.000000
1,female,35.000000
0,male,54.000000
1,female,58.000000
1,male,28.000000


It is possible to extend a table with more columns possibly as a result from a computation in other columns

To create a new column, just assign an expression to a new column name e.g.

```python
df["above_average"] = (df.score > df.score.mean())
```

In [11]:
countries = pd.read_csv("datasets/WorldDBTables/CountryTable.csv")
countries.columns

Index(['code', 'name', 'continent', 'region', 'surface_area',
       'independence_year', 'population', 'life_expectancy', 'gnp', 'gnp_old',
       'local_name', 'government_form', 'head_of_state', 'capital', 'code2'],
      dtype='object')

### Exercise
calculate the population density of each country

The countries table contains the population size in the `population` column and the land extension in the `surface area`
1. calculate the ratio of these two columns and store it in a new column called `population density`

In [12]:
countries["population_density"] = countries.population /  countries.surface_area
countries.loc[:,["name","population_density"]].head()

name,population_density
Aruba,533.678756
Afghanistan,34.841816
Angola,10.329670
Anguilla,83.333333
Albania,118.310839


2. sort the table in descending order using the `.sort_values` function
3. restrict the columns to only the `["name","population_density"]` columns
4. show the first lines of the table using the `.head()` method: what are the most densely populated countries?

In [13]:
countries.sort_values("population_density",ascending=False).loc[:,["name","population_density"]].head()

name,population_density
Macao,26277.777778
Monaco,22666.666667
Hong Kong,6308.837209
Singapore,5771.844660
Gibraltar,4166.666667


## Join and concatenation

A relation may be composed by more than a table; this may offer some consistency and operation efficiency.

If two tables represents entitites which are related they can be **joined** by selecting one or more columns which contains those attributes which creates the relationship.

Per each matched rows in a table, this is replicated as many times as the matched rows in the other table

There are four kinds of available joins

| join | data incuded | added missing values |
|------|--------------|----------------|
| inner | only rows which belongs to both tables | none |
| left | all rows of the first table | for all non matching rows in the first table |
| right | all rows of the second table | for all non matching rows in the second table |
| outer | all rows of both tables | for all non matching rows |

Suppose we have a list of courses, classrooms and classroom booking per each course; if we want to know where each professor should hold his lesson we need to join these tables

| course_id | title | professor |
|-----------|-------|-----------|
| 1 | quantum field theory | Bohr |
| 2 | thermodynamics | Carnot |
| 3 | statistics | Gosset |

| classroom_id | building | floor |
|--------------|----------|-------|
| p124 | Purple | 1 |
| r201 | Red | 2 |

| course_id | classroom_id | weekday | start | end |
|-----------|--------------|---------|-------|-----|
| 1 | p124 | Monday | 9 | 11 |
| 1 | r201 | Wednesday | 14 | 15 |
| 2 | r201 | Tuesday | 14 | 17 |
| 3 | r201 | Monday | 14 | 15 |
| 3 | p124 | Tuesday | 9 | 10 |
| 3 | p124 | Wednesday | 9 | 10 |

the `pd.merge()` function performs the join operation e.g.
```python
courses_classrooms = pd.merge(courses,classroom)
courses_bookings = ps.merge(courses_classroom, bookings)
```
The default kind of join is `inner` you can use the `how=` optional argument to choose another kind.

`pd.merge` will join by default all columns with identical name: if you want to restrict the join to a given list of column you can use the `on=` option.

If you have different names for the join columns you can use `left_on=` and `right_on=` options to match them.

### Exercise
- in the country table we have a list of countries including their population
- in the languages table we have a list of languages spoken in each country and the percentage of the population which speaks said language
- in the country table we have a textual `code` which is uniquely assigned to each county
- in the languages table we have the same code in a column called `country_code`

1. load the language table from `datasets/WorldDBTables/LanguageTable.csv` using the `pd.read_csv` function and store it in a variable called `languages`
2. create a table named `language_by_country` using the `pd.merge` function and joining the column `code` of table `countries` with the column `country_code` from the `languages` table
3. calculate the number of people speaking a language by multiplying the `population` column with the `percentage` column (don't forget to divide by 100!); put the result in a column called `poeple_speaking`
4. show some lines of the table keeping only the following columns: `["name","language","people_speaking","official"]` what do you see?

In [14]:
languages = pd.read_csv("datasets/WorldDBTables/LanguageTable.csv")

In [15]:
languages_by_country = pd.merge(
    countries, languages, 
    how="inner", 
    left_on=["code"], right_on=["country_code"]
)
languages_by_country["people_speaking"] = languages_by_country.population * \
    languages_by_country.percentage / 100
languages_by_country[["name","language","people_speaking","official"]].head()

name,language,people_speaking,official
Aruba,Dutch,5459.000000,T
Aruba,English,9785.000000,F
Aruba,Papiamento,79001.000000,F
Aruba,Spanish,7622.000000,F
Afghanistan,Balochi,204480.000000,F


### Concatenation
It may happen that your data is collected in separated dataframes with the same columns ans you need to create a single one from all of them.

In [13]:
#!unzip ROMA.zip TG_SOUID100860.txt

In [14]:
#!unzip BARI.zip TG_SOUID245914.txt

In [16]:
roma = pd.read_csv("TG_SOUID100860.txt",skiprows=20)
roma.head()

SOUID,DATE,TG,Q_TG
100860,19510101,76,0
100860,19510102,108,0
100860,19510103,116,0
100860,19510104,115,0
100860,19510105,82,0


In [45]:
bari = pd.read_csv("TG_SOUID245914.txt",skiprows=20)
bari.head()

,SOUID,DATE,TG,Q_TG
0,245914,20211201,-9999,9
1,245914,20211202,-9999,9
2,245914,20211203,-9999,9
3,245914,20211204,-9999,9
4,245914,20211205,-9999,9


the `pd.concat()` function can concatenate a list of data frames; the default behavior is consistent with the semantic of relations and it retunrns a single data frame:
- columns will be the union of all columns of each individual data frame in the input
- rows will keep the same order as the data frames

In [18]:
temperatures = pd.concat([roma,bari])
temperatures.columns = list(map(str.strip,temperatures.columns))
for col in ["SOUID","Q_TG"]:
    temperatures[col] = temperatures[col].astype("category")
temperatures["DATE"]=pd.to_datetime(temperatures["DATE"],format="%Y%m%d")
print(temperatures.Q_TG.unique())
temperatures.loc[temperatures.Q_TG != 9,:].describe(include="all")

[0, 9, 1]
Categories (3, int64): [0, 1, 9]


,SOUID,DATE,TG,Q_TG
count,21717.0,21717,21717.000000,21717.0
unique,2.0,NaN,NaN,2.0
top,100860.0,NaN,NaN,0.0
freq,21525.0,NaN,NaN,21711.0
mean,NaN,1980-11-11 03:17:47.716535360,154.883732,NaN
min,NaN,1951-01-01 00:00:00,-56.000000,NaN
25%,NaN,1965-11-17 00:00:00,101.000000,NaN
50%,NaN,1980-09-28 00:00:00,150.000000,NaN
75%,NaN,1995-08-22 00:00:00,212.000000,NaN
max,NaN,2022-10-18 00:00:00,327.000000,NaN


## Aggregation
very often you may want to group your data according to one or more attribute and perform some calculation on each group, this operation is called **aggregation**

e.g. suppose I want to split a restaurant bill with my friends and I have a dataframe which looks like the following table

| person | item | amount |
|--------|------|--------|
| me | pepperoni pizza | 12 |
| me | lager pils | 5 |
| andrea | cheeseburger | 10 |
| andrea | coca cola | 2 |
| andrea | french fries | 2 |

```python
groups = bill.groupby(["person"])
groups.agg({"amount":"sum"})
```

will return

| person | amount |
|--------|--------|
| me | 17 |
| andrea | 14 |

It is also possible to ask for multiple aggregation by using a list of functions

In [23]:
g = titanic.groupby(["Pclass","Sex"])
age_summary = g.agg({"Age":["min","max","mean"]})
age_summary

Age                 
                min   max       mean
Pclass Sex                          
1      female  2.00  63.0  34.611765
       male    0.92  80.0  41.281386
2      female  2.00  57.0  28.722973
       male    0.67  70.0  30.740707
3      female  0.75  63.0  21.750000
       male    0.42  74.0  26.507589

Please note that here the generated columns are accessible using a tuple i.e.
```python
age_summary[("Age","mean")]
```

## Exercise
using the `languages_by_country` table we created in the previous exercise
1. create a grouping by using the `"language"` column
2. using the `.agg()` method calculate how many people speak each language
3. sort the dataset from the largest group descending
4. show the first lines using `.head()` method

In [19]:
g = languages_by_country.groupby(["language"])
languages_spoken = g.agg({"people_speaking":"sum"})
languages_spoken_sorted = languages_spoken.sort_values("people_speaking",ascending=False)
languages_spoken_sorted.head(20)

,people_speaking
language,
Chinese,1.190153e+09
Hindi,4.056192e+08
Spanish,3.079974e+08
Bengali,2.093047e+08
Arabic,2.054908e+08
Portuguese,1.769819e+08
Japanese,1.262540e+08
Punjabi,1.040254e+08
English,9.161603e+07


In [20]:
g = languages_by_country.groupby(["continent","language"])
languages_spoken = g.agg({"people_speaking":"sum"})
languages_spoken_sorted = languages_spoken.sort_values("people_speaking",ascending=False)
languages_spoken_sorted.head(20)

people_speaking
continent     language                   
Asia          Chinese        1.189353e+09
              Hindi          4.051690e+08
              Bengali        2.093047e+08
South America Portuguese     1.660380e+08
              Spanish        1.456209e+08
Africa        Arabic         1.343921e+08
North America Spanish        1.327070e+08
Asia          Japanese       1.255736e+08
              Punjabi        1.038073e+08
              Javanese       8.357016e+07
              Telugu         7.906564e+07
              Marathi        7.501099e+07
              Korean         7.144569e+07
              Vietnamese     6.990842e+07
              Arabic         6.918428e+07
              Tamil          6.868227e+07
              Urdu           6.358947e+07
Europe        English        6.179907e+07
              French         6.045545e+07
              Italian        5.534415e+07

In [21]:
languages_spoken_sorted=languages_spoken_sorted.reset_index()

In [21]:
g = languages_spoken_sorted.groupby(["continent"])
result = []
for i,subtable in g:
    result.append(subtable.head(3).reset_index())
pd.concat(result).head()

continent,language,people_speaking
Africa,Arabic,134392131.000000
Africa,Hausa,29225396.000000
Africa,Joruba,24868874.000000
Asia,Chinese,1189353427.000000
Asia,Hindi,405169038.000000


## Translate the content of a table
Suppose we need to translate some foreign language content

In [22]:
resources = pd.read_csv("ds523_consumoacquaenergia.csv",sep=";")
resources.head()

anno,Consumo pro capite tipo,Consumo pro capite
2011,Energia elettrica per uso domestico,1196.100000
2011,Gas metano per uso domestico e riscaldamento,377.900000
2011,Acqua fatturata per uso domestico,83.100000
2010,Energia elettrica per uso domestico,1200.700000
2010,Gas metano per uso domestico e riscaldamento,406.200000


The second column looks like a categorical series, so let's check it

In [4]:
resources["Consumo pro capite tipo"].unique()

array(['Energia elettrica per uso domestico',
       'Gas metano per uso domestico e riscaldamento',
       'Acqua fatturata per uso domestico'], dtype=object)

we can pass a dictionary to the `.map()` method like this:

In [23]:
translate = {
    'Energia elettrica per uso domestico':'electricity',
    'Gas metano per uso domestico e riscaldamento':'methan',
    'Acqua fatturata per uso domestico':'water'
}
resources["type"] = resources["Consumo pro capite tipo"].map(translate)

Also columns can be renamed or removed

In [24]:
resources = resources.rename({"anno":"year","Consumo pro capite":"usage per person"}, axis="columns")
del resources["Consumo pro capite tipo"]

In [25]:
resources.head()

year,usage per person,type
2011,1196.100000,electricity
2011,377.900000,methan
2011,83.100000,water
2010,1200.700000,electricity
2010,406.200000,methan


## Pivoting and melting
Pivot is a family of aggregation functions whose main purpose is to collect data from a relation and aggregate them by using one or more attribute columns.

This process will create a column per each combination of the attributes; the result table is sometime referred as "wide format" table or "two entries table"; let's make an example

In [26]:
resources2 = resources.pivot(index="year",columns="type",values="usage per person").reset_index()
resources2

year,electricity,methan,water
2000,1130.200000,509.000000,92.100000
2001,1143.900000,500.700000,91.300000
2002,1195.500000,504.200000,90.400000
2003,1222.800000,480.200000,87.300000
2004,1228.600000,442.400000,80.400000
2005,1225.000000,434.500000,81.300000
2006,1219.700000,431.300000,82.200000
2007,1197.000000,381.100000,81.600000
2008,1203.000000,384.900000,84.500000
2009,1202.900000,389.600000,85.800000


As there was exactly one value per each year and each commodity the previous example just moved values without performing any calculation.

Suppose now we want to split some restaurant bill

In [27]:
bill = pd.DataFrame([
    {"item":"pepperoni pizza", "amount":12, "customer": "Marco", "day": "Monday"},
    {"item":"beer", "amount":7.5, "customer": "Marco", "day": "Monday"},
    {"item":"coffee", "amount":1.2, "customer": "Marco", "day": "Monday"},
    {"item":"pizza margherita", "amount":10, "customer": "Luca", "day": "Monday"},
    {"item":"wine", "amount":10, "customer": "Luca", "day": "Monday"},
    {"item":"steak", "amount":20, "customer": "Marco", "day": "Tuesday"},
    {"item":"bottled water", "amount":5, "customer": "Marco", "day": "Tuesday"},
])
bill

item,amount,customer,day
pepperoni pizza,12.000000,Marco,Monday
beer,7.500000,Marco,Monday
coffee,1.200000,Marco,Monday
pizza margherita,10.000000,Luca,Monday
wine,10.000000,Luca,Monday
steak,20.000000,Marco,Tuesday
bottled water,5.000000,Marco,Tuesday


pandas function `pivot_table` allows to define an aggregation function in case of collision

In [29]:
splitted_bill = pd.pivot_table(bill,index="day",values="amount",columns="customer",aggfunc="sum")
splitted_bill

customer,Luca,Marco
day,,
Monday,20.0,20.7
Tuesday,NaN,25.0


pandas `pd.melt()` function provides a way to get a "long format" table

In [30]:
pd.melt(splitted_bill)

customer,value
Luca,20.000000
Luca,nan
Marco,20.700000
Marco,25.000000
